## SQL Queries and some SQL Feature Engineering:

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
cnx = create_engine('postgresql://localhost:5432/telcochurn')

In [3]:
pd.read_sql_query('''SELECT * FROM churntable6 LIMIT 5''', cnx)

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn,autopayment,longtermcontract
0,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,No,No
1,5380-WJKOV,Male,0,No,No,34,Yes,Yes,Fiber optic,No,...,Yes,Yes,Month-to-month,Yes,Electronic check,106.35,3549.25,Yes,No,No
2,3714-NTNFO,Female,0,No,No,49,Yes,Yes,Fiber optic,No,...,No,Yes,Month-to-month,Yes,Electronic check,84.50,3906.7,No,No,No
3,9763-GRSKD,Male,0,Yes,Yes,13,Yes,No,DSL,Yes,...,No,No,Month-to-month,Yes,Mailed check,49.95,587.45,No,No,No
4,1066-JKSGK,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,Month-to-month,No,Mailed check,20.15,20.15,Yes,No,No


In [4]:
pd.read_sql_query('''select count(churn) from churntable6
where churn = 'No';''', cnx)

,count
0,5174


In [5]:
pd.read_sql_query('''select count(churn) from churntable6
where churn = 'Yes';''', cnx)

,count
0,1869


In [6]:
pd.read_sql_query('''select count(churn) from churntable6
group by churn;''', cnx)

,count
0,5174
1,1869


imbalanced classes - will want to use class imbalance techniques

--groupby customers that have both phone and internet service:

In [7]:
pd.read_sql_query('''select churn, count(churn) from churntable6
where phoneservice = 'Yes' and internetservice != 'No'
group by churn;''', cnx)

,churn,count
0,No,3249
1,Yes,1586


3,249 stayed and 1,586 left (of customers that have both phone and internet service)

--groupby customers that have only phone service (no internet service):

In [8]:
pd.read_sql_query('''select churn, count(churn) from churntable6
where phoneservice = 'Yes' and internetservice = 'No'
group by churn;''', cnx)

,churn,count
0,No,1413
1,Yes,113


1413 stayed and 113 left. Good indicator that there is something with the internetservice that customers do not like

-- groupby customers that have only internet service (no phone service)

In [9]:
pd.read_sql_query('''select churn, count(churn) from churntable6
where phoneservice = 'No' and internetservice != 'No'
group by churn;''', cnx)

,churn,count
0,No,512
1,Yes,170


In [10]:
pd.read_sql_query('''select churn, internetservice ,count(churn) from churntable6
where phoneservice = 'Yes'
group by churn, internetservice;''', cnx)

,churn,internetservice,count
0,No,No,1413
1,No,DSL,1450
2,Yes,No,113
3,Yes,Fiber optic,1297
4,No,Fiber optic,1799
5,Yes,DSL,289


1450 with DSL stayed; 289 with DSL left.

1799 with Fiber Optic stayed; 1297 with Fiber Optic Left.

1413 with no internet stayed; 113 with no internet left. 

-- what do people with high tenure have in common?

In [11]:
pd.read_sql_query('''select churn, internetservice, count(churn)
from churntable6
where tenure>=12
group by churn, internetservice;''', cnx)

,churn,internetservice,count
0,No,No,1023
1,No,DSL,1543
2,Yes,No,23
3,Yes,Fiber optic,678
4,Yes,DSL,169
5,No,Fiber optic,1538


high tenure with low rates of churn had either no internet or DSL

In [12]:
pd.read_sql_query('''select churn, internetservice, count(churn)
from churntable6
where tenure<12
group by churn, internetservice;''', cnx)

,churn,internetservice,count
0,No,No,390
1,No,DSL,419
2,Yes,Fiber optic,619
3,Yes,No,90
4,No,Fiber optic,261
5,Yes,DSL,290


highest rates of churn for low tenure were Fiber Optic and then DSL */

--whats the average monthlycost of customers with different types of internet service?

In [13]:
pd.read_sql_query('''select internetservice, avg(monthlycharges) as avgmonth
from churntable6
group by internetservice;''', cnx)

,internetservice,avgmonth
0,No,21.079194
1,DSL,58.102169
2,Fiber optic,91.500129


In [14]:
pd.read_sql_query('''select internetservice, avg(monthlycharges) as avgmonth
from churntable6
where phoneservice != 'No'
group by internetservice;''', cnx)

,internetservice,avgmonth
0,No,21.079194
1,DSL,64.405894
2,Fiber optic,91.500129


In [15]:
pd.read_sql_query('''select internetservice, avg(monthlycharges) as avgmonth
from churntable6
where phoneservice = 'No'
group by internetservice;''', cnx)

,internetservice,avgmonth
0,DSL,42.028592


DSL is the only internet service type where phoneservice='No'

Average monthly cost of phone service only (no internet service):

In [16]:
pd.read_sql_query('''select phoneservice, avg(monthlycharges) as avgmonth
from churntable6
where internetservice = 'No'
group by phoneservice;''', cnx)

,phoneservice,avgmonth
0,Yes,21.079194


-- does streaming tv and/or movies have an affect on churn?

In [17]:
pd.read_sql_query('''select churn, streamingtv, streamingmovies, count(churn)
from churntable6
where internetservice != 'No'
group by churn, streamingtv, streamingmovies;''', cnx)

,churn,streamingtv,streamingmovies,count
0,Yes,No,Yes,247
1,No,No,No,1323
2,No,Yes,Yes,1369
3,No,No,Yes,545
4,Yes,Yes,Yes,571
5,Yes,No,No,695
6,No,Yes,No,524
7,Yes,Yes,No,243


people that either did not get any type of streaming services or people that got both movies and tv appeared to be the most satified */

-- does paperless billing have an affect on churn?

In [18]:
pd.read_sql_query('''select churn, paperlessbilling, count(churn)
from churntable6
group by churn, paperlessbilling;''', cnx)

,churn,paperlessbilling,count
0,No,Yes,2771
1,Yes,Yes,1400
2,No,No,2403
3,Yes,No,469


paperlessbilling=no had much lower churn rate than paperlessbilling=yes */

--does auto payment methods have an affect on churn?

In [19]:
pd.read_sql_query('''select churn, paymentmethod, count(churn)
from churntable6
group by churn, paymentmethod;''', cnx)

,churn,paymentmethod,count
0,Yes,Credit card (automatic),232
1,No,Electronic check,1294
2,Yes,Electronic check,1071
3,No,Bank transfer (automatic),1286
4,Yes,Bank transfer (automatic),258
5,No,Mailed check,1304
6,No,Credit card (automatic),1290
7,Yes,Mailed check,308


/* automatic payment methods had much lower churn rate than non-automatic methods.
  * Electronic check method in particular, had the highest churn rate */

-- add feature engineered columns (separate payment method into autopayment (yes/no); longtermcontract (yes/no):

**These columns were added in DBeaver so cannot be re-added in sqlalchemy. Here is the code I used to feature engineer regardless:**

In [20]:
# pd.read_sql_query('''alter table churntable6 
# 	add column autopayment varchar(5),
# 	add column longtermcontract varchar(5);''', cnx)

In [21]:
# pd.read_sql_query('''update churntable6 
# set autopayment = 'Yes'
# where paymentmethod = 'Bank transfer (automatic)';''', cnx)

In [22]:
# pd.read_sql_query('''update churntable6 
# set autopayment = 'Yes'
# where paymentmethod = 'Credit card (automatic)';''', cnx)

In [23]:
# pd.read_sql_query('''update churntable6 
# set autopayment = 'No'
# where paymentmethod = 'Electronic check';''', cnx)

In [24]:
# pd.read_sql_query('''update churntable6 
# set autopayment = 'No'
# where paymentmethod = 'Mailed check';''', cnx)

-- update longtermcontract values:

In [25]:
# pd.read_sql_query('''update churntable6 
# set longtermcontract = 'No'
# where contract = 'Month-to-month';''', cnx)

In [26]:
# pd.read_sql_query('''update churntable6 
# set longtermcontract = 'Yes'
# where contract = 'One year';''', cnx)

In [27]:
# pd.read_sql_query('''update churntable6 
# set longtermcontract = 'Yes'
# where contract = 'Two year';''', cnx)

--check that columns are added correctly:

In [28]:
pd.read_sql_query('''select * from churntable6 limit 5;''', cnx)

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn,autopayment,longtermcontract
0,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,No,No
1,5380-WJKOV,Male,0,No,No,34,Yes,Yes,Fiber optic,No,...,Yes,Yes,Month-to-month,Yes,Electronic check,106.35,3549.25,Yes,No,No
2,3714-NTNFO,Female,0,No,No,49,Yes,Yes,Fiber optic,No,...,No,Yes,Month-to-month,Yes,Electronic check,84.50,3906.7,No,No,No
3,9763-GRSKD,Male,0,Yes,Yes,13,Yes,No,DSL,Yes,...,No,No,Month-to-month,Yes,Mailed check,49.95,587.45,No,No,No
4,1066-JKSGK,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,Month-to-month,No,Mailed check,20.15,20.15,Yes,No,No


In [29]:
pd.read_sql_query('''select * from churntable6 
where contract != 'Month-to-month'
limit 5;''', cnx)

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn,autopayment,longtermcontract
0,6967-QIQRV,Male,0,Yes,Yes,15,Yes,No,Fiber optic,Yes,...,Yes,Yes,One year,No,Electronic check,101.90,1667.25,No,No,Yes
1,6461-PPAXN,Female,0,Yes,Yes,35,Yes,No,DSL,Yes,...,No,No,One year,Yes,Bank transfer (automatic),54.95,1916,No,Yes,Yes
2,1154-HYWWO,Male,0,No,No,38,Yes,No,DSL,Yes,...,Yes,No,One year,No,Mailed check,70.60,2708.2,No,No,Yes
3,8091-TTVAX,Male,0,Yes,No,58,Yes,Yes,Fiber optic,No,...,Yes,Yes,One year,No,Credit card (automatic),100.35,5681.1,No,Yes,Yes
4,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,One year,No,Mailed check,56.95,1889.5,No,No,Yes


-- how does longtermcontract affect churn

In [30]:
pd.read_sql_query('''select churn, longtermcontract, count(churn)
from churntable6
group by churn, longtermcontract;''', cnx)

,churn,longtermcontract,count
0,No,Yes,2954
1,Yes,Yes,214
2,No,No,2220
3,Yes,No,1655


customers with a longtermcontract had much lower churn rate than those with month-to-month contract */

-- how does autopayment affect churn

In [31]:
pd.read_sql_query('''select churn, autopayment, count(churn)
from churntable6
group by churn, autopayment;''', cnx)

,churn,autopayment,count
0,No,Yes,2576
1,Yes,Yes,490
2,No,No,2598
3,Yes,No,1379


customers with autopayment methods had much lower churn rate than non-autopayment methods